# XLM-R Fine-Tuning

## Load the Dataset

In [1]:
from datasets import load_dataset

emotion = load_dataset("emotion")
label_names = emotion["train"].features["label"].names
id2label = {i: l for i, l in enumerate(label_names)}
label2id = {l: i for i, l in enumerate(label_names)}

## Define the Model Architecture

In [2]:
model_ckpt = "xlm-roberta-base"

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [4]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
from transformers import AutoModelForSequenceClassification

model = (
    AutoModelForSequenceClassification
    .from_pretrained(model_ckpt, num_labels=len(label_names),
                     id2label=id2label, label2id=label2id)
    .to(device)
)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Define Quality Metrics

In [6]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(model_output):
    true_ids = model_output.label_ids
    predicted_ids = model_output.predictions.argmax(-1)
    f1 = f1_score(true_ids, predicted_ids, average='weighted')
    acc = accuracy_score(true_ids, predicted_ids)

    return {'accuracy': acc, 'f1': f1}

# Tokenize Dataset

In [7]:
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

In [8]:
emotion_tokenized = emotion.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

# Train the Model

In [9]:
import os
from transformers import Trainer, TrainingArguments

batch_size = 64
logging_steps = len(emotion_tokenized['train']) // batch_size
model_name = f"{model_ckpt}-finetuned-emotion"
output_dir = os.path.join('../models', model_name)
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    evaluation_strategy='epoch',
    disable_tqdm=False,
    logging_steps=logging_steps,
    push_to_hub=False,
    save_strategy='epoch',
    load_best_model_at_end=True,
    log_level='error',
    seed=254
)

trainer = Trainer(model=model, args=training_args, compute_metrics=compute_metrics,
                  train_dataset=emotion_tokenized['train'],
                  eval_dataset=emotion_tokenized['validation'],
                  tokenizer=tokenizer)

In [10]:
trainer.train()

  0%|          | 0/750 [00:00<?, ?it/s]

KeyboardInterrupt: 